# Validation script for XPower EMA calculation during the ride

#### Dataset to validate was copied from Clark_Rationale_2013s.xls which contains original data and examples from paper (tab '7 - Quantify workout')

Rationale and resources for teaching the mathematical modeling of athletic
training and performance
David C. Clarke1 and Philip F. Skiba2
1
Department of Biological Engineering, Massachusetts Institute of Technology, Cambridge, Massachusetts; and 2
College
of Life and Environmental Sciences, University of Exeter, Exeter, United Kingdom
Submitted 2 August 2011; accepted in final form 1 February 2013

In [23]:
# import libraries
import pandas as pd
import numpy as np

# copy from clipboard to pandas DatFrame
df = pd.read_clipboard()

In [24]:
# Calculate a 25 second rolling average. Note this can also be done at once without the merge

sma = df.power.rolling(window=25).mean().to_frame().rename(columns=lambda x: "25s_sma_watts")
df = pd.merge(df, sma, left_index=True, right_index=True, how='left')

In [25]:
# We fill the zero values in our 25s SMA so we can easily define our first valid rolling 25s value starting point. Since these NaNs only appear in the first 25 secs of the files this is not a problem
# Inspired and modified from: https://stackoverflow.com/questions/36923494/pandas-dataframe-use-previous-row-value-for-complicated-if-conditions-to-deter

df['25s_sma_watts'] =  df['25s_sma_watts'].fillna(0)

# define period. For xPower this is 25

n = 25

def apply_func_decorator(func):
    prev_row = {}
    def wrapper(curr_row, **kwargs):
        val = func(curr_row, prev_row)
        prev_row.update(curr_row)
        prev_row['xPower'] = val
        return val
    return wrapper

@apply_func_decorator
def running_total(curr_row, prev_row):
    return np.where(prev_row.get('25s_sma_watts')==0, curr_row['25s_sma_watts'], (curr_row['25s_sma_watts'] - prev_row.get('xPower', 0))*(2/(n+1)) + prev_row.get('xPower', 0))

df['xPower'] = df.apply(running_total, axis=1)

#### You will see that the xPower is the same as the EMA which is calculated in the Excel tab cel G42

In [26]:
df.iloc[23:27,:]

,power,EMA,25s_sma_watts,xPower
23,0,NaN,0.00,0.00
24,4,"78,84",78.84,78.84
25,0,"78,84",78.84,78.84
26,0,"78,84",78.84,78.84


In [27]:
df['Xpower4'] = df.xPower**4
df.iloc[23:26,:]

,power,EMA,25s_sma_watts,xPower,Xpower4
23,0,NaN,0.00,0.00,0.000000e+00
24,4,"78,84",78.84,78.84,3.863549e+07
25,0,"78,84",78.84,78.84,3.863549e+07


In [28]:
print('Xpower is', np.mean(df.Xpower4)**0.25)

Xpower is 277.9837880287564
